In [1]:
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
x, y = make_blobs(random_state=777) # 인위적인 데이터셋 생성
# 데이터와 타깃 레이블을 훈련 세트와 테스트 세트로 나눕니다
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
logreg = LogisticRegression(solver='lbfgs', max_iter=10000) #모델 생성
logreg = logreg.fit(x_train, y_train) # 훈련 세트로 학습
print("테스트 세트 점수: {:.2f}".format(logreg.score(x_test, y_test))) # 평가

테스트 세트 점수: 0.96


In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
iris = load_iris()
logreg = LogisticRegression(max_iter=1000)
scores = cross_val_score(logreg, iris.data, iris.target, cv=5)
print("교차 검증 점수: {}".format(scores))
print("교차 검증 평균 점수: {:.2f}".format(scores.mean()))

교차 검증 점수: [0.96666667 1.         0.93333333 0.96666667 1.        ]
교차 검증 평균 점수: 0.97


교차검증 데이터를 나눌때 데이터의 구조상 균일하게 나누어지지 않을 수 있음

In [4]:
iris['target']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [5]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3)
scores = cross_val_score(logreg, iris.data, iris.target, cv=kfold)
print("교차 검증 점수:{}".format(scores))

교차 검증 점수:[0. 0. 0.]


그래서 shuffle=True 라는 옵션을 줌

In [6]:
kfold = KFold(n_splits=3, shuffle=True, random_state=777)
scores = cross_val_score(logreg, iris.data, iris.target, cv=kfold)
print("교차 검증 점수:{}".format(scores))

교차 검증 점수:[1.   0.9  0.98]


데이터가 적을때

In [7]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(logreg, iris.data, iris.target, cv=loo)
print("교차 검증 분할 횟수: ", len(scores))
print("평균 정확도: {:.2f}".format(scores.mean()))

교차 검증 분할 횟수:  150
평균 정확도: 0.97


각 데이터에 그룹을 지정해서 훈련 세트와 테스트 세트에 같은 그룹으로
지정된 데이터가 포함되지 않도록 분할

사진에서 표정을 인식하는 시스템을 만들기 위해 100명에 대한 n개의
사진을 모은 경우 → 각 사람을 그룹으로 지정해서 같은 사람의 사진이
훈련 세트와 테스트 세트에 동시에 포함되지 않도록 분할

In [8]:
from sklearn.model_selection import ShuffleSplit
shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)
scores = cross_val_score(logreg, iris.data, iris.target,
cv=shuffle_split)
print("교차 검증 점수:\n{}".format(scores))
print("평균 정확도: {:.2f}".format(scores.mean()))

교차 검증 점수:
[0.97333333 0.92       0.93333333 0.93333333 0.93333333 0.93333333
 0.93333333 0.93333333 0.97333333 0.98666667]
평균 정확도: 0.95


In [9]:
from sklearn.model_selection import GroupKFold
# 인위적 데이터셋 생성
X, y = make_blobs(n_samples=12, random_state=0)
# 처음 세 개의 샘플은 같은 그룹에 속하고, 다음은 네 개의 샘플이 같습니다.
groups = [0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3]
scores = cross_val_score(logreg, X, y, groups = groups, cv=GroupKFold(n_splits=3))
print("교차 검증 점수:\n{}".format(scores))

교차 검증 점수:
[0.75       0.6        0.66666667]


그리드 서치 (Grid Search) <br>
일반화 성능을 최대로 높여주는 모델의 매개변수를 찾는 과정이 필요한데
가장 널리 사용하는 매개변수 탐색 방법 중 하나가 Grid Search

In [10]:
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score

# 데이터셋 로드 및 분할
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)

best_score = 0
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # 매개변수의 각 조합에 대해 SVC를 훈련시킵니다
        svm = SVC(gamma=gamma, C=C)
        scores = cross_val_score(svm, X_train, y_train, cv=3)  # 교차 검증 점수 계산
        score = scores.mean()  # 교차 검증 평균 점수 계산

        if score > best_score:  # 점수가 더 높으면 매개변수와 함께 기록
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}

print("최고 점수: {:.2f}".format(best_score))
print("최적 파라미터: {}".format(best_parameters))


최고 점수: 0.96
최적 파라미터: {'C': 100, 'gamma': 0.01}


In [11]:
from sklearn.svm import SVC
# 데이터를 훈련+검증 세트 그리고 테스트 세트로 분할
X_trainval, X_test, y_trainval, y_test = train_test_split(
iris.data, iris.target, random_state=0)
# 훈련+검증 세트를 훈련 세트와 검증 세트로 분할
X_train, X_valid, y_train, y_valid = train_test_split(
X_trainval, y_trainval, random_state=1)
print("훈련 세트의 크기: {} 검증 세트의 크기: {} 테스트 세트의 크기:"
" {}\n".format(X_train.shape[0], X_valid.shape[0], X_test.shape[0]))

훈련 세트의 크기: 84 검증 세트의 크기: 28 테스트 세트의 크기: 38



In [12]:
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score

# 데이터셋 로드 및 분할
iris = load_iris()
X_trainval, X_test, y_trainval, y_test = train_test_split(iris['data'], iris['target'], test_size=0.2, random_state=42)

best_score = 0
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # 매개변수의 각 조합에 대해 SVC를 훈련
        svm = SVC(gamma=gamma, C=C)
        scores = cross_val_score(svm, X_trainval, y_trainval, cv=5)
        score = scores.mean()

        if score > best_score:  # 점수가 더 높으면 기록
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}

# 최적의 매개변수로 최종 모델 훈련 및 평가
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)
test_score = svm.score(X_test, y_test)

print("검증 세트에서 최고 점수: {:.2f}".format(best_score))
print("최적 파라미터: ", best_parameters)
print("최적 파라미터에서 테스트 세트 점수: {:.2f}".format(test_score))


검증 세트에서 최고 점수: 0.96
최적 파라미터:  {'C': 100, 'gamma': 0.01}
최적 파라미터에서 테스트 세트 점수: 1.00


In [13]:
best_score = 0
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # 매개변수의 각 조합에 대해 SVC를 훈련시킵니다
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        score = svm.score(X_valid, y_valid)  # 검증 세트로 SVC를 평가합니다

        if score > best_score:  # 점수가 더 높으면 매개변수와 함께 기록합니다
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}

# 훈련 세트와 검증 세트를 합쳐 모델을 다시 만든 후 테스트 세트를 사용해 평가
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)
test_score = svm.score(X_test, y_test)

print("검증 세트에서 최고 점수: {:.2f}".format(best_score))
print("최적 파라미터: ", best_parameters)
print("최적 파라미터에서 테스트 세트 점수: {:.2f}".format(test_score))


검증 세트에서 최고 점수: 0.96
최적 파라미터:  {'C': 10, 'gamma': 0.001}
최적 파라미터에서 테스트 세트 점수: 0.97


In [14]:
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC

# 데이터 로드
iris = load_iris()

# 딕셔너리 형식의 매개변수 그리드 만들기
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
print("매개변수 그리드:\n{}".format(param_grid))

# GridSearchCV 객체 만들기
grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)

# 그리드 서치 수행
grid_search.fit(X_train, y_train)

# 테스트 세트 점수 출력
print("테스트 세트 점수: {:.2f}".format(grid_search.score(X_test, y_test)))


매개변수 그리드:
{'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
테스트 세트 점수: 0.97


In [15]:
grid_search.best_estimator_

SVC(C=10, gamma=0.1)

In [16]:
param_grid = [
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'gamma': [0.001, 0.01, 0.1, 1, 10, 100]
    },
    {
        'kernel': ['linear'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100]
    }
]

grid_search = GridSearchCV(
    SVC(),
    param_grid,
    cv=5,
    return_train_score=True
)

grid_search.fit(X_train, y_train)

print("최적 파라미터: {}".format(grid_search.best_params_))
print("최고 교차 검증 점수: {:.2f}".format(grid_search.best_score_))


최적 파라미터: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
최고 교차 검증 점수: 0.97


평가 지표와 측정 <br>
불균형 데이터 세트 : 많은쪽 데이터를 잘 맞추도록 최적화 됨
* 보통 우리는 적은 데이터를 맞추고 싶음